<a href="https://colab.research.google.com/github/pandian-raja/EVA8/blob/main/Session_2_5_PyTorch_101_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

randomNumber = torch.tensor([0,1,2,3,4,5,6,7,8,9])

In [44]:
class TestDataSet(Dataset):
    def __init__(self):
        self.data = torchvision.datasets.MNIST('/content/mnist', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
        
        self.random = torch.randint(0, 10, (len(self.data),))
        
    def __getitem__(self, index):
        image, label = self.data[index]
        randomInput = randomNumber.eq(self.random[index]).long()
        return image, label, randomInput

    def __len__(self):
        return len(self.data)

In [45]:
class TrainDataSet(Dataset):
    def __init__(self):
        self.data = torchvision.datasets.MNIST('/content/mnist', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
        self.random = torch.randint(0, 10, (len(self.data),))
        
    def __getitem__(self, index):
        image, label = self.data[index]
        randomInput = randomNumber.eq(self.random[index]).long()
        return image, label, randomInput

    def __len__(self):
        return len(self.data)

In [46]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        #3x3: SIZE { in = 28 , out= 26}, recptive field = 3, CHANNEL { in = 1, out= 32 }
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3)
        #3x3: SIZE { in = 26, out= 24}, recptive field = 5, CHANNEL { in = 32, out= 64 }
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        #3x3: SIZE { in = 24, out = 22},  recptive field = 7, CHANNEL { in = 64, out= 128 }
        self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        
        #MX: SIZE { in = 22, out = 11},  recptive field = 14, TODO: CHANNEL { in = 64, out= 128 }

        #3x3: SIZE { in = 11, out = 9},  recptive field = 16, CHANNEL { in = 64, out= 128 }
        self.conv4 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3)
        #F: SIZE { in = 24, out = 22},  TODO: recptive field = 7, CHANNEL { in = 64, out= 128 }
        self.fc1 = nn.Linear(in_features=128*9*9,out_features=64)
        #F: SIZE { in = 24, out = 22},  TODO: recptive field = 7, CHANNEL { in = 64, out= 128 }
        self.fc2 = nn.Linear(in_features=64,out_features=10)
        
        self.fc3 = nn.Linear(in_features=10,out_features=20)

        self.out = nn.Linear(in_features=20,out_features=19)

    def forward(self, image, random):

        x = image
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) 
        x = self.conv4(x)
        x = F.relu(x)
        x = x.reshape(-1, 128*9*9)
        # x = x.reshape(1, -1)
        x = self.fc1(x)
        x = F.relu(x)
        mnist_output = self.fc2(x)
        x = mnist_output+random
        x = self.fc3(x)
        x = F.relu(x)
        x = self.out(x)
        mnist_output = F.softmax(mnist_output, dim=1)
        x = F.softmax(x, dim=1)
        return mnist_output, x

In [47]:
def get_num_correct(images, labels, random, random_label):
    
    return images.argmax(dim=1).eq(labels).sum().item(), random.argmax(dim=1).eq(random_label).sum().item()


In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network  = Network()
network.to(device)
testDataset = TestDataSet()
trainDataset = TrainDataSet()
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=200, shuffle= True)
trainDataLoader = torch.utils.data.DataLoader(trainDataset, batch_size=200, shuffle= True)
optimizer = optim.Adam(network.parameters(), lr=0.001)
totalEpoch = 10
for epoch in range(totalEpoch):
    total_mnist_loss = 0
    total_mnist_correct = 0
    total_random_correct = 0
    total_random_loss = 0
    for batch in trainDataLoader:
        images, labels, randoms = batch
        images, labels, randoms = images.to(device), labels.to(device), randoms.to(device)
        random_labels = labels+randoms.argmax(dim=1)
        mnist_predict, random_predict = network(images, randoms)
        mnist_loss = F.cross_entropy(mnist_predict, labels)
        random_loss = F.cross_entropy(random_predict, random_labels)
        total_mnist_loss += mnist_loss.item()
        total_random_loss += random_loss.item()
        loss = mnist_loss+random_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_mnist_correct, total_random_correct = get_num_correct(mnist_predict,labels,random_predict,random_labels)


    print(
        "epoch:", epoch, 
        "mnist_correct:", total_mnist_correct, 
        "mnist_loss: %.2f" % total_mnist_loss,
        "random_correct:", total_random_correct, 
        "random_loss: %.2f" % total_random_loss,
        "total_correct:", total_mnist_correct+total_random_correct, 
        "loss: %.2f" % (total_mnist_loss+total_random_loss)
    )



epoch: 0 mnist_correct: 186 mnist_loss: 489.80 random_correct: 20 random_loss: 873.82 total_correct: 206 loss: 1363.61
epoch: 1 mnist_correct: 182 mnist_loss: 470.93 random_correct: 6 random_loss: 869.69 total_correct: 188 loss: 1340.62
epoch: 2 mnist_correct: 198 mnist_loss: 467.25 random_correct: 24 random_loss: 868.08 total_correct: 222 loss: 1335.32
epoch: 3 mnist_correct: 198 mnist_loss: 442.55 random_correct: 23 random_loss: 867.27 total_correct: 221 loss: 1309.82
epoch: 4 mnist_correct: 200 mnist_loss: 441.58 random_correct: 25 random_loss: 865.30 total_correct: 225 loss: 1306.88
epoch: 5 mnist_correct: 199 mnist_loss: 441.02 random_correct: 29 random_loss: 863.19 total_correct: 228 loss: 1304.21
epoch: 6 mnist_correct: 199 mnist_loss: 440.58 random_correct: 27 random_loss: 860.11 total_correct: 226 loss: 1300.69
epoch: 7 mnist_correct: 199 mnist_loss: 440.39 random_correct: 33 random_loss: 856.33 total_correct: 232 loss: 1296.71
epoch: 8 mnist_correct: 199 mnist_loss: 440.28 ra

In [53]:
print("loss: %.2f" % (total_mnist_loss+total_random_loss))

loss: 1384.87


In [22]:
for epoch in range(totalEpoch):
    total_mnist_loss = 0
    total_mnist_correct = 0
    total_random_correct = 0
    total_random_loss = 0
    for batch in trainDataLoader:
        images, labels, randoms, x = batch
        images, labels, randoms = images.to(device), labels.to(device), randoms.to(device)
        random_labels = labels+randoms.argmax(dim=1)
        mnist_predict, random_predict = network(images, randoms)
        mnist_loss = F.cross_entropy(mnist_predict, labels)
        random_loss = F.cross_entropy(random_predict, random_labels)
        total_mnist_loss += mnist_loss.item()
        total_random_loss += random_loss.item()
        loss = mnist_loss+random_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_mnist_correct, total_random_correct = get_num_correct(mnist_predict,labels,random_predict,random_labels)


    print(
        "epoch:", epoch, 
        "mnist_correct:", total_mnist_correct, 
        "mnist_loss: %.2f" % total_mnist_loss,
        "random_correct:", total_random_correct, 
        "random_loss: %.2f" % total_random_loss,
        "total_correct:", total_mnist_correct+total_random_correct, 
        "loss: %.2f" % total_mnist_loss+total_random_loss
    )

epoch: 0 mnist_correct: 198 mnist_loss: 440.13707852363586 random_correct: 30 random_loss: 853.687361240387 total_correct: 228 loss: 1293.8244395256042
epoch: 1 mnist_correct: 200 mnist_loss: 440.1806298494339 random_correct: 41 random_loss: 849.637229681015 total_correct: 241 loss: 1289.817862033844
epoch: 2 mnist_correct: 198 mnist_loss: 440.00813949108124 random_correct: 42 random_loss: 845.2159383296967 total_correct: 240 loss: 1285.22407579422
epoch: 3 mnist_correct: 198 mnist_loss: 440.0135688781738 random_correct: 51 random_loss: 841.6334455013275 total_correct: 249 loss: 1281.6470112800598
epoch: 4 mnist_correct: 200 mnist_loss: 439.9158339500427 random_correct: 49 random_loss: 838.7114663124084 total_correct: 249 loss: 1278.6272983551025
epoch: 5 mnist_correct: 198 mnist_loss: 439.8821151256561 random_correct: 49 random_loss: 835.4232578277588 total_correct: 247 loss: 1275.3053755760193
epoch: 6 mnist_correct: 200 mnist_loss: 439.89011669158936 random_correct: 50 random_loss: 

In [41]:
network = Network()
testDataset = TestDataSet()
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=2, shuffle= True)
batch = next(iter(testDataLoader))

In [513]:
preds = network(batch[0], batch[2])
loss1 = F.cross_entropy(preds[0], batch[1])
loss2 = F.cross_entropy(preds[1], batch[3])
loss = loss2+loss1
print(loss1.item(),loss2.item(), get_num_correct(preds, batch[1], batch[2]))
optimizer.zero_grad()
loss.backward()
optimizer.step()

1.828912615776062 2.748772382736206 (63, 28)


In [ ]:

# for epoch in range(totalEpoch):
#     total_loss = 0
#     total_correct = 0
#     mnist_loss = 0
#     random_loss = 0
#     for batch in trainDataLoader:
#         images, labels, randoms, x = batch
#         images, labels, randoms = images.to(device), labels.to(device), randoms.to(device)
#         random_labels = labels+randoms.argmax(dim=1)
#         mnist_predict, random_predict = network(images, randoms)
#         mnist_loss = F.cross_entropy(mnist_predict, labels)
#         random_loss = F.cross_entropy(random_predict, random_labels)
#         loss = mnist_loss+random_loss
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#         total_correct += sum(get_num_correct(mnist_predict,labels,random_predict,random_labels))


#     print(
#         "epoch:", epoch, 
#         "total_correct:", total_correct, 
#         "loss:", total_loss
#     )